In [140]:
import pandas as pd
import numpy as np

In [141]:
ROOT = '/home/robert/DataSpellProjects/marchMadness/'

In [142]:
ELO_RATING = {}

## Import Data

In [143]:
RegularSeason = pd.read_csv(f'{ROOT}/Data/MRegularSeasonDetailedResultsModified.csv', index_col=False)
Tourney = pd.read_csv(f'{ROOT}/Data/MNCAATourneyCompactResults.csv')
TeamIDs = pd.read_csv(f'{ROOT}/Data/MTeams.csv')
Rankings = pd.read_csv(f'{ROOT}/Data/MMasseyOrdinals.csv')
RegularSeason = RegularSeason[RegularSeason.Season==2021]
RegularSeason = RegularSeason[['Season', 'DayNum', 'WTeamID', 'LTeamID', 'WScore', 'LScore']]
RegularSeason['WTeamElo'] = np.nan
RegularSeason['LTeamElo'] = np.nan

## Elo Functions

In [144]:
K = 30
def win_chance(elo1, elo2):
    """Given elo1 and elo2, will return the chance of elo1 winning"""
    return  1 / ( 1 + 10**((elo2-elo1)/400) )

def update_elo(winner_elo, loser_elo):
    """Given elo1 (winner) and elo2 (loser) will return new elos"""
    elo_change = K * (1 - win_chance(winner_elo, loser_elo))
    new_elo1 = int(winner_elo + elo_change)
    new_elo2 = int(loser_elo - elo_change)
    return new_elo1, new_elo2
def margin_of_victory(elo_win, elo_loss, score_win, score_loss):
    return (np.log((score_win-score_loss)+1) * 2.2) / ((elo_win - elo_loss)*0.001 + 2.2)

In [145]:
def get_team_ids():
    team_ids = []
    WTeamIDs = RegularSeason['WTeamID'].unique().tolist()
    LTeamIDs = RegularSeason['LTeamID'].unique().tolist()
    for team in WTeamIDs:
        team_ids.append(team)
    for team in LTeamIDs:
        if team not in WTeamIDs:
            team_ids.append(team)
    return team_ids
team_ids = get_team_ids()

In [146]:
def generate_elo():
    for team in team_ids:
        ELO_RATING[team] = 1500
generate_elo()
RegularSeason

,Season,DayNum,WTeamID,LTeamID,WScore,LScore,WTeamElo,LTeamElo
92832,2021,23,1101,1190,70,47,NaN,NaN
92833,2021,23,1104,1240,81,57,NaN,NaN
92834,2021,23,1111,1354,81,61,NaN,NaN
92835,2021,23,1113,1348,94,88,NaN,NaN
92836,2021,23,1114,1341,71,66,NaN,NaN
...,...,...,...,...,...,...,...,...
96682,2021,132,1104,1261,80,79,NaN,NaN
96683,2021,132,1159,1259,85,72,NaN,NaN
96684,2021,132,1222,1153,91,54,NaN,NaN
96685,2021,132,1228,1326,91,88,NaN,NaN


In [147]:
for i in RegularSeason.index:
    MoV = margin_of_victory(ELO_RATING[RegularSeason.at[i, 'WTeamID']],ELO_RATING[RegularSeason.at[i, 'WTeamID']], RegularSeason.at[i, 'WScore'], RegularSeason.at[i, 'LScore'])
    WTeamElo, LTeamElo = update_elo(ELO_RATING[RegularSeason.at[i, 'WTeamID']], ELO_RATING[RegularSeason.at[i, 'LTeamID']])
    RegularSeason.at[i, 'WTeamElo'] = WTeamElo + MoV
    RegularSeason.at[i, 'LTeamElo'] = LTeamElo - MoV
    ELO_RATING[RegularSeason.at[i, 'WTeamID']] = WTeamElo + MoV
    ELO_RATING[RegularSeason.at[i, 'LTeamID']] = LTeamElo - MoV

In [148]:
def get_team_id(team_name):
    return TeamIDs.loc[TeamIDs['TeamName']==team_name]['TeamID'].values[0]

In [149]:
TeamIDs

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2022
1,1102,Air Force,1985,2022
2,1103,Akron,1985,2022
3,1104,Alabama,1985,2022
4,1105,Alabama A&M,2000,2022
...,...,...,...,...
367,1468,Bellarmine,2021,2022
368,1469,Dixie St,2021,2022
369,1470,Tarleton St,2021,2022
370,1471,UC San Diego,2021,2022


In [155]:
win_chance(ELO_RATING[get_team_id('USC')], ELO_RATING[get_team_id('Oregon')])

0.4947996470685843